In [26]:
from bs4 import BeautifulSoup
import html2text

converter = html2text.HTML2Text()

def segment_html(html: str) -> list[str]:
    # Convert markdown to HTML
    # Parse HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Iterate through the elements and associate headers with content
    headers = ['title', 'h1', 'h2']
    segments = []
    current_header = None
    for element in soup.children:
        if element.name in headers:
            current_header = element.getText()
            segments.append(element.getText())
        elif current_header:
            content = converter.handle(str(element))
            segments[-1] = segments[-1] + " " + content
    return segments

In [28]:
import fitz

def classify_text(text, font_size):
    if font_size >= 18:
        return f"<h1>{text}</h1>"
    else:
        return f"<p>{text}</p>"

def pdf_to_html_with_headers(pdf_path):
    doc = fitz.open(pdf_path)
    html_content = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            if block["type"] == 0:
                for line in block["lines"]:
                    for span in line["spans"]:
                        text = span["text"].strip()
                        
                        # Skip spans that are just a bullet point
                        if text == "-":
                            continue

                        font_size = span["size"]
                        html_text = classify_text(text, font_size)
                        html_content.append(html_text)

    doc.close()
    return "".join(html_content)




pdf_path = 'notes.pdf'
html_output = pdf_to_html_with_headers(pdf_path)
print(segment_html(html_output))

with open('output.html', 'w') as html_file:
    html_file.write(html_output)

['Anatomy Course Notes', 'Introduction to Anatomy \n\nDefinition:\n\n \n\nStudy of the structure of body parts and their relationships.\n\n \n\nBranches:\n\n \n\nGross/Macroscopic, Microscopic, Developmental\n\n', 'The Skeletal System \n\nFunctions:\n\n \n\nSupport, movement, protection, blood cell production, calcium storage.\n\n \n\nBone Types:\n\n \n\nLong (femur), Short (carpals), Flat (sternum), Irregular (vertebrae)\n\n', 'Muscular System \n\nMuscle Types:\n\n \n\nSkeletal (voluntary movement), Smooth (organ walls), Cardiac (heart).\n\n \n\nFunctions:\n\n \n\nMovement, posture, joint stability, heat production.\n\n', 'Nervous System \n\nComponents:\n\n \n\nBrain, spinal cord, nerves.\n\n \n\nFunctions:\n\n \n\nRegulates and controls body functions, sensory response, motor response.\n\n']


In [ ]:
%pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip uninstall aspose-pdfcd 

^C
Note: you may need to restart the kernel to use updated packages.


In [5]:

import aspose.pdf as ap

input_pdf = "notes.pdf"
output_pdf = "convert_pdf_to_html.html"
# Open PDF document
document = ap.Document(input_pdf)

# save document in HTML format
save_options = ap.HtmlSaveOptions()
document.save(output_pdf, save_options)